In [4]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('C:/Users/Sharwari Prakash M/Dropbox/My PC (DESKTOP-48H0BCO)/Downloads/dt.csv')
df.head()

# Define the calculate entropy function
def calculate_entropy(df_label):
    classes, class_counts = np.unique(df_label, return_counts=True)
    entropy_value = np.sum(
        [(-class_counts[i] / np.sum(class_counts)) * np.log2(class_counts[i] / np.sum(class_counts)) for i in
         range(len(classes))])
    return entropy_value

# Define the calculate information gain function
def calculate_information_gain(dataset, feature, label):
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])
    values, feat_counts = np.unique(dataset[feature], return_counts=True)

    # Calculate the weighted feature entropy
    weighted_feature_entropy = np.sum(
        [(feat_counts[i] / np.sum(feat_counts)) * calculate_entropy(
            dataset.where(dataset[feature] == values[i]).dropna()[label]) for i in range(len(values))])
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

# Define the create decision tree function
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])

    if len(unique_data) <= 1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]

    elif len(features) == 0:
        return parent

    else:
        parent = unique_data[np.argmax(datum[1])]

        # Call the calculate_information_gain function
        item_values = [calculate_information_gain(dataset, feature, label) for feature in features]

        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature: {}}

        features = [i for i in features if i != optimum_feature]

        for value in np.unique(dataset[optimum_feature]):
            min_data = dataset.where(dataset[optimum_feature] == value).dropna()

            # Recursive call to create_decision_tree function
            min_tree = create_decision_tree(min_data, df, features, label, parent)

            decision_tree[optimum_feature][value] = min_tree

        return decision_tree

# Define the predict function
def predict_diabetes(test_data,decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]


        prediction = 0
        if type(decision_tree) is dict:
            # Recusrsive call to predict_diabetes function
            prediction = predict_diabetes(test_data, decision_tree)
        else:
            prediction = decision_tree
            break;

    return prediction

# Set the features and label
features = df.columns[:-1]
label = 'diabetes'
parent = None

# Train the decision tree model
decision_tree = create_decision_tree(df, df, features, label, parent)

# Predict using the trained model
sample_data = {'glucose': 77, 'bloodpressure': 112}
test_data = pd.Series(sample_data)

prediction = predict_diabetes(test_data, decision_tree)
print(prediction)


0.0


In [7]:
# Load the test data (assuming you have a separate CSV file for testing)
test_df = pd.read_csv('C:/Users/Sharwari Prakash M/Dropbox/My PC (DESKTOP-48H0BCO)/Downloads/te.csv')




# Extract the features and label for the test data
test_features = test_df.drop(label, axis=1)
test_labels = test_df[label]

# Function to predict a single label
def predict_label(sample, decision_tree):
    if isinstance(decision_tree, dict):
        feature = list(decision_tree.keys())[0]
        value = sample[feature]
        if value in decision_tree[feature]:
            sub_tree = decision_tree[feature][value]
            return predict_label(sample, sub_tree)
        else:
            return "Unknown"
    else:
        return decision_tree

# Predict labels for the test data using the trained decision tree
test_predictions = test_features.apply(lambda x: predict_label(x, decision_tree), axis=1)

# Calculate accuracy by comparing predicted labels with actual labels
accuracy = (test_predictions == test_labels).mean() * 100
print(f"Accuracy: {accuracy}%")



Accuracy: 0.0%


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('C:/Users/Sharwari Prakash M/Dropbox/My PC (DESKTOP-48H0BCO)/Downloads/s.csv')

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Convert the gender column to numerical values
df['gender_encoded'] = label_encoder.fit_transform(df['Gender'])

# Print the updated dataset
print(df)

   Gender  Age  gender_encoded
0    Male   10               1
1  Female   45               0
2    Male   34               1
3  Female   78               0
4   Other   90               2
